<a href="https://colab.research.google.com/github/oktaviacitra/data-mining/blob/main/MidTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from google.colab import drive

# Connect to Google Drive

In [ ]:
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Data Mining/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1

In [ ]:
dataset = pd.read_csv(path + "titanic.csv")
dataset.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


# 2

In [ ]:
data = dataset.loc[:, ['Sex', 'Pclass', 'Age', 'Fare', 'Survived']]
data.shape

(891, 5)

# 3

In [ ]:
pos_missing = [index for index, row in data.iterrows() if row.isnull().any()]

In [ ]:
train_data = data.loc[:, ['Sex', 'Pclass', 'Fare', 'Age', 'Survived']].dropna().drop(columns=['Age'])
train_data.shape

(714, 4)

# 4

In [ ]:
train_label = data.loc[:, ['Age']].dropna().astype(int)
train_label.shape

(714, 1)

# 5

In [ ]:
test_data = data.iloc[pos_missing].drop(columns=['Age'])
test_data.head(3)

,Sex,Pclass,Fare,Survived
5,male,3,8.4583,0
17,male,2,13.0000,1
19,female,3,7.2250,1


In [ ]:
train_data['Sex'] = train_data['Sex'].replace({'female':1, 'male':0})
test_data['Sex'] = test_data['Sex'].replace({'female':1, 'male':0})

# Data Normalization

In [ ]:
new_min, new_max = 0, 1
def train_min_max(input_data, column_name):
  new_min_max = []
  data = input_data[column_name]
  min = data.min()
  max = data.max()
  new_min_max.append(min)
  new_min_max.append(max)
  input_data[column_name] = (data - min) * (new_max - new_min) / (max - min) + new_min
  return new_min_max, input_data

In [ ]:
def test_min_max(input_data, new_min_max, column_name):
  data = input_data[column_name]
  min = new_min_max[0]
  max = new_min_max[1]
  input_data[column_name] = (data - min) * (new_max - new_min) / (max - min) + new_min
  return input_data

# 6

In [ ]:
new_min_max, train_data = train_min_max(train_data, 'Fare')
train_data

,Sex,Pclass,Fare,Survived
0,0,3,0.014151,0
1,1,1,0.139136,1
2,1,3,0.015469,1
3,1,1,0.103644,1
4,0,3,0.015713,0
...,...,...,...,...
885,1,3,0.056848,0
886,0,2,0.025374,0
887,1,1,0.058556,1
889,0,1,0.058556,1


# 7

In [ ]:
test_data = test_min_max(test_data, new_min_max, 'Fare')
test_data

,Sex,Pclass,Fare,Survived
5,0,3,0.016510,0
17,0,2,0.025374,1
19,1,3,0.014102,1
26,0,3,0.014102,0
28,1,3,0.015379,1
...,...,...,...,...
859,0,3,0.014110,0
863,1,3,0.135753,0
868,0,3,0.018543,0
878,0,3,0.015412,0


# 8

In [ ]:
kNN = KNeighborsClassifier(n_neighbors=3, weights='distance')
kNN.fit(train_data, train_label)
class_result = kNN.predict(test_data)
class_result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


array([21, 31, 15, 25, 19, 28, 58, 22, 20, 28, 34, 40, 22,  2, 28, 40, 20,
       28, 34, 22, 34, 34, 28, 25,  4, 34, 32,  4,  9, 20, 17, 28, 22, 64,
       24, 16, 23, 31,  1, 32, 22, 28, 32, 28, 10, 21, 14,  1, 20, 26, 32,
       21, 56, 22, 28, 64, 40, 27, 22,  1, 30, 34, 39, 28,  4, 40, 28, 24,
       71, 25, 19, 19, 40, 13, 22, 24, 28, 21, 10, 28, 19, 28, 23, 28, 20,
       32, 24, 16, 18, 34, 35, 32, 34, 28, 21, 20, 31, 28, 10, 18, 26, 40,
       21, 28, 34, 24, 25, 17, 27, 17,  4, 19,  0, 20, 27, 32, 34, 23, 17,
       22, 17, 17, 34, 21,  6, 25, 28, 36, 20,  1, 32, 21, 38, 21, 32, 42,
       28, 19, 28, 18, 35, 28, 23, 32, 16, 20, 56, 40, 16, 28, 28, 28, 26,
       19, 36, 24, 25, 32, 21, 24, 32, 16, 56, 38, 45, 28, 29, 17, 34, 26,
       28, 33, 17, 16, 16, 28, 10])

# 9

In [ ]:
data['Age'] = data['Age'].fillna(-1).astype(int)
c = 0
for i in range(0, len(data['Age']) - 1):
  if data['Age'][i] == -1:
    data['Age'][i] = class_result[c]
    c += 1
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Sex,Pclass,Age,Fare,Survived
0,male,3,22,7.2500,0
1,female,1,38,71.2833,1
2,female,3,26,7.9250,1
3,female,1,35,53.1000,1
4,male,3,35,8.0500,0
...,...,...,...,...,...
886,male,2,27,13.0000,0
887,female,1,19,30.0000,1
888,female,3,10,23.4500,0
889,male,1,26,30.0000,1


# 10

In [ ]:
test_dataset = pd.read_csv(path + "titanic_test.csv")
test_dataset

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


# 11

In [ ]:
train_data = data.loc[:, ['Sex', 'Pclass', 'Fare', 'Age']]
train_data

,Sex,Pclass,Fare,Age
0,male,3,7.2500,22
1,female,1,71.2833,38
2,female,3,7.9250,26
3,female,1,53.1000,35
4,male,3,8.0500,35
...,...,...,...,...
886,male,2,13.0000,27
887,female,1,30.0000,19
888,female,3,23.4500,10
889,male,1,30.0000,26


# 12

In [ ]:
train_label = data.loc[:, ['Survived']]
train_label

,Survived
0,0
1,1
2,1
3,1
4,0
...,...
886,0
887,1
888,0
889,1


# 13

In [ ]:
test_data = test_dataset.loc[:, ['Sex', 'Pclass', 'Fare', 'Age']]

In [ ]:
pos_missing = [index for index, row in test_data.iterrows() if row.isna().any()]
test_data = test_data.dropna()
test_data.shape

(331, 4)

# 14

In [ ]:
test_label = pd.read_csv(path + "titanic_testlabel.csv").drop(pos_missing)
test_label.shape

(331, 2)

# 15

In [ ]:
new_min_max_fare, train_data = train_min_max(train_data, 'Fare')

In [ ]:
new_min_max_age, train_data = train_min_max(train_data, 'Age')
train_data

,Sex,Pclass,Fare,Age
0,male,3,0.014151,0.2750
1,female,1,0.139136,0.4750
2,female,3,0.015469,0.3250
3,female,1,0.103644,0.4375
4,male,3,0.015713,0.4375
...,...,...,...,...
886,male,2,0.025374,0.3375
887,female,1,0.058556,0.2375
888,female,3,0.045771,0.1250
889,male,1,0.058556,0.3250


# 16

In [ ]:
test_data = test_min_max(test_data, new_min_max_fare, 'Fare')

In [ ]:
test_data = test_min_max(test_data, new_min_max_age, 'Age')
test_data

,Sex,Pclass,Fare,Age
0,male,3,0.015282,0.43125
1,female,3,0.013663,0.58750
2,male,2,0.018909,0.77500
3,male,3,0.016908,0.33750
4,female,3,0.023984,0.27500
...,...,...,...,...
409,female,3,0.026887,0.03750
411,female,1,0.175668,0.46250
412,female,3,0.015176,0.35000
414,female,1,0.212559,0.48750


# 17

In [ ]:
train_data['Sex'] = train_data['Sex'].replace({'female':1, 'male':0})
test_data['Sex'] = test_data['Sex'].replace({'female':1, 'male':0})

In [ ]:
kNN.fit(train_data, train_label)
pred_label = kNN.predict(test_data)
pred_label

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0,

# 18

In [ ]:
error_total = sum(i != j for i, j in zip(pred_label, test_label['Survived']))
print("Error Total: ", error_total)

Error Total:  70


# 19

In [ ]:
error_ratio = (error_total / test_data.shape[0]) * 100
print("Error Ratio : ", error_ratio, "%")

Error Ratio :  21.148036253776432 %
